In [14]:
import os

import pandas as pd
from pynwb import NWBFile
from datetime import datetime
import h5py as h5
import numpy as np
import xarray as xr

# load preprocessed data

In [ ]:
# Dataset is saved to disk using the Dataset.to_netcdf method:
# ds.to_netcdf('saved_on_disk.nc')
# ds_disk = xr.open_dataset('saved_on_disk.nc')

In [4]:
basepath = '/Users/xiaoxuanj/work/work_allen/Ephys/ecephys_pipeline_test/input_json/'

In [16]:
electrodes = pd.read_csv(os.path.join(basepath, 'electrodes.csv'))
sorted_units = pd.read_csv(os.path.join(basepath, 'sorted_units.csv'))

In [23]:
big_table = pd.merge(sorted_units, electrodes, 
                     left_on='peak_channel_id', right_on='id', 
                     how='outer', 
                     suffixes=['_sorted_unit', '_channel'], 
                     copy=True)

In [25]:
big_table.sort_values('unit_id')

,cell_type,peak_channel_id,snr,unit_id,x_pos_sorted_unit,y_pos_sorted_unit,id,is_reference,local_channel,probe,x_pos_channel,y_pos_channel
0,noise,93.0,0.334085,0.0,1.0,-2920.0,93.0,False,93.0,probeC,1.0,-2920.0
15,noise,121.0,2.218261,1.0,1.0,-2640.0,121.0,False,121.0,probeC,1.0,-2640.0
18,fs,22.0,3.628959,2.0,2.0,-3620.0,22.0,False,22.0,probeC,2.0,-3620.0
20,fs,35.0,1.993670,3.0,1.0,-3500.0,35.0,False,35.0,probeC,1.0,-3500.0
31,fs,111.0,1.656102,4.0,1.0,-2740.0,111.0,False,111.0,probeC,1.0,-2740.0
45,noise,103.0,0.775735,5.0,1.0,-2820.0,103.0,False,103.0,probeC,1.0,-2820.0
58,noise,91.0,1.565008,6.0,1.0,-2940.0,91.0,False,91.0,probeC,1.0,-2940.0
65,noise,75.0,0.457389,7.0,1.0,-3100.0,75.0,True,75.0,probeC,1.0,-3100.0
66,noise,4.0,1.551839,8.0,2.0,-3800.0,4.0,False,4.0,probeC,2.0,-3800.0
68,noise,109.0,1.705037,9.0,1.0,-2760.0,109.0,False,109.0,probeC,1.0,-2760.0


In [5]:
ds_disk = xr.open_dataset(basepath+'spike_times.nc')

In [13]:
ds_disk['spike_times'].values

KeyboardInterrupt: 

In [10]:
ds_disk.dims

Frozen(SortedKeysDict({'unit_id': 4012, 'sample': 325631}))

# create nwb instance and add info to general

In [10]:
nwb2 = NWBFile('NWB 1.0 > NWB 2.0 converter',        # source
                  str(nwb1['session_description'].value), # session description
                  str(nwb1['identifier'].value),          # identifier
                  str(nwb1['session_start_time'].value.astype(str)),  # date
                  experimenter = nwb1['general']['experimenter'].value.astype(str),
                  lab = nwb1['general']['lab'].value.astype(str),
                  institution = nwb1['general']['lab'].value.astype(str),
                  experiment_description = nwb1['general']['experiment_description'].value.astype(str),
                  session_id='SESSION_ID')

In [11]:
device = nwb2.create_device(name='Neuropixels Prototype Rig', source="a source")

In [12]:
probes = [p for p in nwb1['processing'].keys() if p[:5] == 'probe']

In [13]:
probes

['probeA', 'probeB', 'probeC', 'probeD', 'probeE', 'probeF']

In [14]:
locations = {'probeA' : 'AM',
             'probeB' : 'PM',
             'probeC' : 'V1',
             'probeD' : 'LM',
             'probeE' : 'AL',
             'probeF' : 'RL'}

xlocs = [16, 48, 0, 32]

In [15]:
for probe in probes:

    electrode_name = probe
    source = "Neuropixels Probe"
    description = "Phase 3a"
    location = locations[probe]

    electrode_group = nwb2.create_electrode_group(electrode_name,
                                                 source=source,
                                                 description=description,
                                                 location=location,
                                                 device=device)
    
    for idx in range(0,384):
        nwb2.add_electrode(idx,
                          x=float(xlocs[idx % 4]), y=float(idx/2*20), z=np.nan,
                          imp=np.nan,
                          location='unknown', filtering='300 Hz highpass',
                          description='channel %s' % idx, group=electrode_group)

In [16]:
list(nwb1['processing']['probeA']['UnitTimes']['86'])

['channel',
 'snr',
 'source',
 'template',
 'times',
 'type',
 'unit_description',
 'waveform',
 'xpos_probe',
 'ypos_probe']

In [17]:
type(list(nwb1['processing']['probeA']['UnitTimes']['86']['times'])[0])

numpy.float64

# add unit times to processing

In [18]:
from pynwb.misc import UnitTimes

module = nwb2.create_processing_module('ecephys', # name
                                       'ecephys', #source
                                        'UnitTimes module' # description
                                      )

for probe in probes:
    units = []
    spike_times=[]
    unittimes = UnitTimes(probe) # source
    for unit in list(nwb1['processing']['probeA']['UnitTimes']):
        if unit!='noise' and unit!= 'unit_list':
            unittimes.add_spike_times(int(unit.encode('utf8')), list(nwb1['processing']['probeA']['UnitTimes'][unit]['times']))
          #  units.append(unit.encode('utf8'))
          #  spike_times.append(list(nwb1['processing']['probeA']['UnitTimes'][unit]['times']))
            
    
    #unittimes = UnitTimes(probe, # source
    #                      unit_ids=units, #unit_ids
    #                      spike_times=spike_times, # list of times for correponding units
    #                      spike_times_index = [], #the indices in spike_times that correspond to each unit in unit_ids
    #                      name='UnitTimes') # name

    module.add_data_interface(unittimes)                               


In [20]:
probe

'probeF'

In [21]:
module.containers

(<pynwb.misc.UnitTimes at 0x108fc8b38>,)

# add stimulus to stimulus group

In [19]:
from pynwb.misc import AbstractFeatureSeries

for stimulus in nwb1['stimulus']['presentation'].keys():
    afs = AbstractFeatureSeries(stimulus, # name
                          stimulus, # source
                          nwb1['stimulus']['presentation'][stimulus]['feature_units'].value, # units
                          nwb1['stimulus']['presentation'][stimulus]['features'].value, # features
                          data = nwb1['stimulus']['presentation'][stimulus]['data'].value, # data
                          timestamps = nwb1['stimulus']['presentation'][stimulus]['timestamps'].value)
    
    nwb2.add_stimulus(afs)

# save file to NWB

In [20]:
from pynwb import NWBHDF5IO

with NWBHDF5IO('/Users/xiaoxuanj/work/work_allen/Ephys/ecephys_pipeline_test/nwb/stimulus.nwb', 'w') as io:
    io.write(nwb2)

# read file with NWB IO

In [23]:
io = NWBHDF5IO('/Users/xiaoxuanj/work/work_allen/Ephys/ecephys_pipeline_test/nwb/stimulus.nwb', 'r')
nwbfile = io.read()

In [27]:
ephys_ts = list(nwbfile.stimulus)

In [28]:
ephys_ts

['flash_250ms_1', 'gabor_20_deg_0', 'perceptual_grouping02_2', 'spontaneous']